pca_w2v_200features 

In [1]:
from google.colab import drive
drive.mount('/content/drive') 

Mounted at /content/drive


In [2]:
import warnings
warnings.filterwarnings("ignore") 

In [3]:
import os 
import numpy as np 
import pandas as pd 
from sklearn import preprocessing
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, classification_report
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.neighbors import KNeighborsClassifier

In [4]:
import pandas as pd
path = "/content/drive/MyDrive/newpro/threeclass_problem/PCA_w2v/data.csv"
df = pd.read_csv(path)
df.head() 

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,Level 5,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,Level 5,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,Level 5,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",Level 5,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,Level 2,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [5]:
df.iloc[:,1].replace(['Level 1', 'Level 2', 'Level 3', 'Level 4', 'Level 5'],
                        [1, 2, 3, 4, 5], inplace=True)

In [6]:
df.head()

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,5,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,5,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,5,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",5,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,2,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [7]:
#define bins
bins = [0,2,4,5]
#name of the group 
group_names = ['Low', 'Moderate', 'High']
df['Level '] = pd.cut(df["Level "],bins,labels = group_names)

In [8]:
df.head(5)

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,High,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,High,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,High,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",High,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,Low,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [9]:
df['Level '] = df['Level '].map({'Low': 1, 'Moderate': 2, 'High': 3}).astype(int)

In [10]:
df.head()

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,3,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,3,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,3,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",3,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,1,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [11]:
import pandas as pd
path = "/content/drive/MyDrive/newpro/threeclass_problem/PCA_w2v/w2v_withstopwordchangelevels_200_features.csv"
x = pd.read_csv(path)
x.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,190,191,192,193,194,195,196,197,198,199
0,0,-1.943060,-3.090360,-0.442178,1.472654,0.603432,-0.901954,2.539306,-0.923905,4.151764,...,-9.095579e-08,4.542387e-08,4.130263e-08,-4.087643e-09,1.254799e-07,7.492882e-08,9.887553e-08,1.070151e-07,1.063767e-07,-1.798151e-08
1,1,-8.527760,3.501452,1.076773,-1.995076,-2.462809,-1.296075,-1.036794,-0.817676,-1.415838,...,9.806283e-08,-5.641690e-08,-4.186269e-08,-9.124331e-08,-1.118461e-07,5.571130e-08,-1.679865e-08,4.446469e-08,-1.007685e-07,7.211234e-08
2,2,-2.953830,-1.749117,3.026401,-2.087218,5.504423,3.526518,4.623261,-1.277347,0.573729,...,-6.411431e-08,9.774892e-08,3.359301e-08,3.067717e-09,1.003161e-07,-1.895610e-08,-1.205588e-07,-1.432922e-08,4.746613e-08,-1.249381e-07
3,3,18.116164,4.904783,4.840255,-3.345236,1.231826,8.072254,-1.968584,0.538603,0.525013,...,-4.665101e-08,7.001322e-08,-3.560619e-08,-7.590926e-08,2.074501e-07,-3.141484e-09,-2.778485e-08,-4.163730e-08,-3.110642e-08,-2.733549e-08
4,4,6.292048,-6.004878,6.499810,2.056146,-4.704546,-2.018001,-4.079839,2.938242,-2.440830,...,-7.975440e-08,-8.125196e-08,1.031441e-07,-2.542697e-07,-5.238057e-08,-4.303173e-08,-8.803244e-08,-1.580093e-09,-2.759144e-09,4.186093e-08


In [12]:
x.drop(x.columns[[0]], axis=1, inplace=True)
x.head()

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,-1.943060,-3.090360,-0.442178,1.472654,0.603432,-0.901954,2.539306,-0.923905,4.151764,-2.120015,...,-9.095579e-08,4.542387e-08,4.130263e-08,-4.087643e-09,1.254799e-07,7.492882e-08,9.887553e-08,1.070151e-07,1.063767e-07,-1.798151e-08
1,-8.527760,3.501452,1.076773,-1.995076,-2.462809,-1.296075,-1.036794,-0.817676,-1.415838,0.577300,...,9.806283e-08,-5.641690e-08,-4.186269e-08,-9.124331e-08,-1.118461e-07,5.571130e-08,-1.679865e-08,4.446469e-08,-1.007685e-07,7.211234e-08
2,-2.953830,-1.749117,3.026401,-2.087218,5.504423,3.526518,4.623261,-1.277347,0.573729,0.437040,...,-6.411431e-08,9.774892e-08,3.359301e-08,3.067717e-09,1.003161e-07,-1.895610e-08,-1.205588e-07,-1.432922e-08,4.746613e-08,-1.249381e-07
3,18.116164,4.904783,4.840255,-3.345236,1.231826,8.072254,-1.968584,0.538603,0.525013,-1.139062,...,-4.665101e-08,7.001322e-08,-3.560619e-08,-7.590926e-08,2.074501e-07,-3.141484e-09,-2.778485e-08,-4.163730e-08,-3.110642e-08,-2.733549e-08
4,6.292048,-6.004878,6.499810,2.056146,-4.704546,-2.018001,-4.079839,2.938242,-2.440830,0.315170,...,-7.975440e-08,-8.125196e-08,1.031441e-07,-2.542697e-07,-5.238057e-08,-4.303173e-08,-8.803244e-08,-1.580093e-09,-2.759144e-09,4.186093e-08


In [13]:
y=df['Level '] 

**Train Test Split**

In [14]:
# Train-Test-Split is performed

from sklearn.model_selection import train_test_split

# Splitting the data into train and test
X_train, X_test, y_train, y_test = train_test_split(x, y, train_size=0.7, random_state=42)
X_train.shape, X_test.shape

((1220, 200), (524, 200))

In [15]:
print(y_train.value_counts(normalize=True)*100)
print("\n")
print(y_test.value_counts(normalize=True)*100)

2    45.327869
1    29.590164
3    25.081967
Name: Level , dtype: float64


2    45.229008
1    29.961832
3    24.809160
Name: Level , dtype: float64


**Model Building**

In [ ]:
def perceptron_classifier (X_train, y_train, X_test, y_test):
  p = Perceptron(random_state=150)
  p.fit(X_train, y_train)
  predictions_train = p.predict(X_train)
  predictions_test = p.predict(X_test)
  train_score = accuracy_score(predictions_train, y_train)
  #print("score on train data: ", train_score)
  test_score = f1_score(predictions_test, y_test, average='weighted' )
  print("Perceptron F1 score:", test_score)

In [ ]:
from sklearn.linear_model import Perceptron
perceptron_classifier(X_train, y_train, X_test, y_test) 

Perceptron F1 score: 0.4301155014523439


**Decision Tree**

In [16]:
clf_dt = DecisionTreeClassifier(max_depth=5, class_weight = "balanced")

In [17]:
clf_dt.fit(X_train,y_train)

DecisionTreeClassifier(class_weight='balanced', max_depth=5)

In [18]:
y_pred_train = clf_dt.predict(X_train)
y_pred_test = clf_dt.predict(X_test)

In [19]:
def classifcation_report_train_test(y_train, y_train_pred, y_test, y_test_pred):

    print('''
            =========================================
               CLASSIFICATION REPORT FOR TRAIN DATA
            =========================================
            ''')
    print(classification_report(y_train, y_train_pred, digits=4))

    print('''
            =========================================
               CLASSIFICATION REPORT FOR TEST DATA
            =========================================
            ''')
    print(classification_report(y_test, y_test_pred, digits=4))

In [20]:
scores = pd.DataFrame(columns=['Model', 'Train_Accuracy', 'Train_Recall', 'Train_Precision', 'Train_F1_Score', 
                               'Test_Accuracy', 'Test_Recall', 'Test_Precision', 'Test_F1_Score'])

def get_metrics(train_actual, train_predicted, test_actual, test_predicted, model_description, dataframe):

    train_accuracy  = accuracy_score(train_actual, train_predicted)
    train_recall    = recall_score(train_actual, train_predicted, average="weighted")
    train_precision = precision_score(train_actual, train_predicted, average="weighted")
    train_f1score   = f1_score(train_actual, train_predicted, average="weighted")
    
    test_accuracy   = accuracy_score(test_actual, test_predicted)
    test_recall     = recall_score(test_actual, test_predicted, average="weighted")
    test_precision  = precision_score(test_actual, test_predicted, average="weighted")
    test_f1score    = f1_score(test_actual, test_predicted, average="weighted")

    dataframe       = dataframe.append(pd.Series([model_description, 
                                                  train_accuracy, train_recall, train_precision, train_f1score,
                                                  test_accuracy, test_recall, test_precision, test_f1score],
                                                 index=scores.columns ), 
                                       ignore_index=True)

    return(dataframe)

In [21]:
scores = get_metrics(y_train, y_pred_train, y_test, y_pred_test, "DecisionTrees", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.448361,0.448361,0.492553,0.443949,0.335878,0.335878,0.387123,0.33561


**Random Forest**

In [22]:
clf_rf = RandomForestClassifier(class_weight = "balanced")

In [23]:
clf_rf.fit(X=X_train, y=y_train)

RandomForestClassifier(class_weight='balanced')

In [24]:
y_pred_train1 = clf_rf.predict(X_train)
y_pred_test1 = clf_rf.predict(X_test)

In [25]:
classifcation_report_train_test(y_train, y_pred_train1, y_test, y_pred_test1)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     1.0000    0.9972    0.9986       361
           2     0.9964    0.9982    0.9973       553
           3     0.9967    0.9967    0.9967       306

    accuracy                         0.9975      1220
   macro avg     0.9977    0.9974    0.9975      1220
weighted avg     0.9975    0.9975    0.9975      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.3333    0.1019    0.1561       157
           2     0.4573    0.8819    0.6023       237
           3     0.1579    0.0231    0.0403       130

    accuracy                         0.4351       524
   macro avg     0.3162    0.3356    0.2662       524
weighted avg     0.3459    0.4351    0.3292       524



In [26]:
scores = get_metrics(y_train, y_pred_train1, y_test, y_pred_test1, "RandomForest", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.448361,0.448361,0.492553,0.443949,0.335878,0.335878,0.387123,0.335610
1,RandomForest,0.997541,0.997541,0.997544,0.997542,0.435115,0.435115,0.345891,0.329177


**AdaBoost**

In [ ]:
clf_adaboost =  AdaBoostClassifier()

In [ ]:
clf_adaboost.fit(X_train, y_train)

AdaBoostClassifier()

In [ ]:
y_pred_train2 = clf_adaboost.predict(X_train)
y_pred_test2 = clf_adaboost.predict(X_test)

In [ ]:

classifcation_report_train_test(y_train, y_pred_train2, y_test, y_pred_test2)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.4912    0.3075    0.3782       361
           2     0.5320    0.7812    0.6330       553
           3     0.5440    0.3235    0.4057       306

    accuracy                         0.5262      1220
   macro avg     0.5224    0.4707    0.4723      1220
weighted avg     0.5229    0.5262    0.5006      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.3222    0.1847    0.2348       157
           2     0.4548    0.6793    0.5448       237
           3     0.2375    0.1462    0.1810       130

    accuracy                         0.3989       524
   macro avg     0.3382    0.3367    0.3202       524
weighted avg     0.3612    0.3989    0.3617       524



In [ ]:
scores = get_metrics(y_train, y_pred_train2, y_test, y_pred_test2, "Adaboost", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.526230,0.526230,0.577394,0.460916,0.429389,0.429389,0.372727,0.352913
1,RandomForest,0.997541,0.997541,0.997550,0.997542,0.442748,0.442748,0.410403,0.341080
2,Adaboost,0.526230,0.526230,0.522920,0.500586,0.398855,0.398855,0.361168,0.361674


**Gradient Boosting Classifier**

In [ ]:
clf_gbm = GradientBoostingClassifier()

In [ ]:
clf_gbm.fit(X=X_train, y=y_train)

GradientBoostingClassifier()

In [ ]:
y_pred_train3 = clf_gbm.predict(X_train)
y_pred_test3 = clf_gbm.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train3, y_test, y_pred_test3)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.9467    0.8366    0.8882       361
           2     0.7983    0.9946    0.8857       553
           3     0.9764    0.6765    0.7992       306

    accuracy                         0.8680      1220
   macro avg     0.9071    0.8359    0.8577      1220
weighted avg     0.8869    0.8680    0.8647      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.2857    0.1783    0.2196       157
           2     0.4553    0.7089    0.5545       237
           3     0.2982    0.1308    0.1818       130

    accuracy                         0.4065       524
   macro avg     0.3464    0.3393    0.3186       524
weighted avg     0.3655    0.4065    0.3617       524



In [ ]:
scores = get_metrics(y_train, y_pred_train3, y_test, y_pred_test3, "GBM", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.526230,0.526230,0.577394,0.460916,0.429389,0.429389,0.372727,0.352913
1,RandomForest,0.997541,0.997541,0.997550,0.997542,0.442748,0.442748,0.410403,0.341080
2,Adaboost,0.526230,0.526230,0.522920,0.500586,0.398855,0.398855,0.361168,0.361674
3,GBM,0.868033,0.868033,0.886870,0.864747,0.406489,0.406489,0.365518,0.361681


**SVM Model**

In [27]:
linear_svm = SVC(kernel='linear', C=1, class_weight = "balanced")

In [28]:
linear_svm.fit(X=X_train, y= y_train)

SVC(C=1, class_weight='balanced', kernel='linear')

In [29]:
y_pred_train4 = linear_svm.predict(X_train)
y_pred_test4 = linear_svm.predict(X_test)

In [30]:
classifcation_report_train_test(y_train, y_pred_train4, y_test, y_pred_test4)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.3667    0.4155    0.3896       361
           2     0.5381    0.4213    0.4726       553
           3     0.3466    0.4281    0.3830       306

    accuracy                         0.4213      1220
   macro avg     0.4171    0.4217    0.4151      1220
weighted avg     0.4394    0.4213    0.4256      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.2778    0.2866    0.2821       157
           2     0.4246    0.3207    0.3654       237
           3     0.2295    0.3231    0.2684       130

    accuracy                         0.3111       524
   macro avg     0.3106    0.3101    0.3053       524
weighted avg     0.3322    0.3111    0.3164       524



In [31]:
scores = get_metrics(y_train, y_pred_train4, y_test, y_pred_test4, "SVM", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.448361,0.448361,0.492553,0.443949,0.335878,0.335878,0.387123,0.335610
1,RandomForest,0.997541,0.997541,0.997544,0.997542,0.435115,0.435115,0.345891,0.329177
2,SVM,0.421311,0.421311,0.439358,0.425588,0.311069,0.311069,0.332200,0.316372


**KNN**

In [ ]:
knn = KNeighborsClassifier(n_neighbors=7)

In [ ]:
knn.fit(X=X_train, y=y_train)

KNeighborsClassifier(n_neighbors=7)

In [ ]:
y_pred_train5 = knn.predict(X_train)
y_pred_test5 = knn.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train5, y_test, y_pred_test5)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.5106    0.5346    0.5223       361
           2     0.5882    0.7360    0.6538       553
           3     0.5667    0.2778    0.3728       306

    accuracy                         0.5615      1220
   macro avg     0.5551    0.5161    0.5163      1220
weighted avg     0.5598    0.5615    0.5444      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.2581    0.2548    0.2564       157
           2     0.4306    0.5232    0.4724       237
           3     0.2593    0.1615    0.1991       130

    accuracy                         0.3531       524
   macro avg     0.3160    0.3132    0.3093       524
weighted avg     0.3364    0.3531    0.3399       524



In [ ]:
scores = get_metrics(y_train, y_pred_train5, y_test, y_pred_test5, "KNN", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.526230,0.526230,0.577394,0.460916,0.429389,0.429389,0.372727,0.352913
1,RandomForest,0.997541,0.997541,0.997550,0.997542,0.442748,0.442748,0.410403,0.341080
2,Adaboost,0.526230,0.526230,0.522920,0.500586,0.398855,0.398855,0.361168,0.361674
3,GBM,0.868033,0.868033,0.886870,0.864747,0.406489,0.406489,0.365518,0.361681
4,SVM,0.455738,0.455738,0.490934,0.293909,0.458015,0.458015,0.348228,0.297618
5,KNN,0.561475,0.561475,0.559809,0.544425,0.353053,0.353053,0.336377,0.339862


**GNB**

In [ ]:
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()

In [ ]:
nb.fit(X=X_train, y=y_train)

GaussianNB()

In [ ]:
y_pred_train6 = nb.predict(X_train)
y_pred_test6 = nb.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train6, y_test, y_pred_test6)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.4246    0.2105    0.2815       361
           2     0.4859    0.7468    0.5887       553
           3     0.3351    0.2092    0.2575       306

    accuracy                         0.4533      1220
   macro avg     0.4152    0.3888    0.3759      1220
weighted avg     0.4299    0.4533    0.4148      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.3088    0.1338    0.1867       157
           2     0.4686    0.7553    0.5784       237
           3     0.3243    0.1846    0.2353       130

    accuracy                         0.4275       524
   macro avg     0.3672    0.3579    0.3334       524
weighted avg     0.3849    0.4275    0.3759       524



In [ ]:
scores = get_metrics(y_train, y_pred_train6, y_test, y_pred_test6, "GNB", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.526230,0.526230,0.577394,0.460916,0.429389,0.429389,0.372727,0.352913
1,RandomForest,0.997541,0.997541,0.997550,0.997542,0.442748,0.442748,0.410403,0.341080
2,Adaboost,0.526230,0.526230,0.522920,0.500586,0.398855,0.398855,0.361168,0.361674
3,GBM,0.868033,0.868033,0.886870,0.864747,0.406489,0.406489,0.365518,0.361681
4,SVM,0.455738,0.455738,0.490934,0.293909,0.458015,0.458015,0.348228,0.297618
5,KNN,0.561475,0.561475,0.559809,0.544425,0.353053,0.353053,0.336377,0.339862
6,GNB,0.453279,0.453279,0.429919,0.414751,0.427481,0.427481,0.384928,0.375886
